In [2]:
import pandas as pd
from collections import defaultdict
from decimal import Decimal, getcontext

# Set precision for Decimal operations (28 digits is default, plenty for financial calculations)
getcontext().prec = 28

# ---------- Display (optional) ----------
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', None)

# ---------- Global State ----------

portfolio_state = {
    'quantities': defaultdict(lambda: Decimal('0')),  # {ticker: quantity} - positive for long, negative for short
    'cost_basis': defaultdict(lambda: Decimal('0')),  # {ticker: absolute cost basis}
    'avg_price': defaultdict(lambda: Decimal('0')),   # {ticker: average entry price}
    'portfolio_df': None                              # accumulator DataFrame
}

COLUMNS = [
    'Ticker',
    'Side',
    'Direction',
    'Current Direction',
    'Quantity Buy',
    'Current Quantity',
    'Price',
    'Avg Price',
    'Cost Basis'
]

# ---------- Lifecycle ----------

def reset_portfolio():
    """
    Reset the portfolio to initial state.
    
    Resets:
        - All portfolio state dictionaries
        - Portfolio DataFrame
    """
    global portfolio_state
    portfolio_state = {
        'quantities': defaultdict(lambda: Decimal('0')),
        'cost_basis': defaultdict(lambda: Decimal('0')),
        'avg_price': defaultdict(lambda: Decimal('0')),
        'portfolio_df': pd.DataFrame(columns=COLUMNS)
    }

def get_portfolio_df():
    """
    Get a copy of the current portfolio DataFrame.
    
    Returns:
        pd.DataFrame: Copy of the portfolio DataFrame with all trade history.
    """
    return portfolio_state['portfolio_df'].copy()

# ---------- Helpers ----------

def normalize_quantity(q):
    """
    Normalize quantity input to Decimal (always returns absolute value).
    
    Accepts:
        - numeric: 10, -5 -> returns Decimal('10'), Decimal('5')
        - strings: "-(-10)" -> Decimal('10'),  "(10)" -> Decimal('10'), "-10" -> Decimal('10')
    
    Args:
        q: Quantity input (int, float, string, or Decimal)
    
    Returns:
        Decimal: Normalized quantity value (always positive/absolute)
    """
    if isinstance(q, Decimal):
        return abs(q)
    if isinstance(q, (int, float)):
        return abs(Decimal(str(q)))
    s = str(q).strip().replace(' ', '')
    if s.startswith('-(') and s.endswith(')'):
        inner = s[2:-1]
        return abs(Decimal(inner))
    if s.startswith('(') and s.endswith(')'):
        inner = s[1:-1]
        return abs(Decimal(inner))
    return abs(Decimal(s))

# ---------- Core Calculations ----------

def calculate_current_quantity_single(ticker, action, q_in, old_quantity):
    """
    Calculate new quantity after trade.
    
    Quantity change formulas:
        - Buy: new_q = old_q + quantity (moves longward)
        - Sell: new_q = old_q - quantity (moves shortward)
        - Hold: new_q = old_q (no change)
    
    Args:
        ticker (str): Ticker symbol
        action (str): Trade action ('buy', 'sell', 'hold')
        q_in (Decimal): Quantity traded
        old_quantity (Decimal): Quantity before trade
    
    Returns:
        Decimal: New quantity after trade (positive for long, negative for short)
    """
    a = str(action).lower()
    qty = abs(q_in) if q_in < 0 else q_in

    if a == 'hold':
        new_q = old_quantity
    elif a == 'buy':
        new_q = old_quantity + qty  # Naturally flips short→long if buying excess
    elif a == 'sell':
        new_q = old_quantity - qty  # Naturally flips long→short if selling excess
    else:
        new_q = old_quantity

    portfolio_state['quantities'][ticker] = new_q
    return new_q

def calculate_avg_price_and_cost_basis_single(ticker, action, price, q_in, old_quantity, new_quantity, old_cost_basis):
    """
    Calculate average price and cost basis for net position using segment-based logic.
    
    Segment Logic:
        - NEW Segment (=1): Opens when old_quantity == 0 and new_quantity != 0
        - SAME Segment (NO): Continues while adding to position (avg cost changes)
        - END Segment (YES): Closes when new_quantity == 0
        - FLIP (YES): Old segment ends, new segment starts with opening price
    
    Cost Basis Formulas by Situation:
        1. Open Long (NEW):     cb = price × qty
        2. Open Short (NEW):    cb = price × qty
        3. Add to Long (NO):    cb = ((old_cost + new_cost) / total_qty) × new_qty
        4. Add to Short (NO):   cb = ((old_cost + new_cost) / total_qty) × abs(new_qty)
        5. Full Close Long (YES):  cb = 0
        6. Full Close Short (YES): cb = 0
        7. Flip Long→Short (YES):  cb = new_short_qty × new_price
        8. Flip Short→Long (YES):  cb = new_long_qty × new_price
    
    Note: Partial closes are handled proportionally
    
    Args:
        ticker (str): Ticker symbol
        action (str): Trade action ('buy', 'sell', 'hold')
        price (Decimal): Trade price
        q_in (Decimal): Quantity traded
        old_quantity (Decimal): Quantity before trade
        new_quantity (Decimal): Quantity after trade
        old_cost_basis (Decimal): Cost basis before trade
    
    Returns:
        tuple: (avg_price, cost_basis) - Both as Decimal (NO PRECISION LOSS)
    """
    a = str(action).lower()
    qty = abs(q_in) if q_in < 0 else q_in
    cb = old_cost_basis

    if a == 'hold':
        # No change
        pass
    
    elif a == 'buy':
        if old_quantity == 0:
            # Case 1: Open Long (NEW segment = 1)
            # Formula: cb = price × qty
            cb = price * qty
            
        elif old_quantity > 0:
            # Case 3: Add to Long (SAME segment, NO change in segment)
            # Formula: cb = ((old_cost + new_cost) / total_qty) × new_qty
            old_cost = old_cost_basis
            new_cost = price * qty
            total_qty = old_quantity + qty
            cb = ((old_cost + new_cost) / total_qty) * new_quantity
            
        elif old_quantity < 0:
            # Buying when short (covering)
            if new_quantity > 0:
                # Case 8: Flip Short→Long (NEW segment, YES change)
                # Formula: cb = new_long_qty × new_price
                new_long_qty = new_quantity
                cb = new_long_qty * price
                
            elif new_quantity == 0:
                # Case 6: Full Close Short (END segment, YES)
                # Formula: cb = 0
                cb = Decimal('0')
                
            else:
                # Still short (partial cover) - proportional reduction
                to_cover = min(qty, abs(old_quantity))
                remaining_short = abs(old_quantity) - to_cover
                if remaining_short > 0:
                    cb = cb * (remaining_short / abs(old_quantity))
                else:
                    cb = Decimal('0')

    elif a == 'sell':
        if old_quantity == 0:
            # Case 2: Open Short (NEW segment = 1)
            # Formula: cb = price × qty
            cb = price * qty
            
        elif old_quantity < 0:
            # Case 4: Add to Short (SAME segment, NO change)
            # Formula: cb = ((old_cost + new_cost) / total_qty) × abs(new_qty)
            old_cost = old_cost_basis
            new_cost = price * qty
            total_qty = abs(old_quantity) + qty
            cb = ((old_cost + new_cost) / total_qty) * abs(new_quantity)
            
        elif old_quantity > 0:
            # Selling when long
            if new_quantity < 0:
                # Case 7: Flip Long→Short (NEW segment, YES change)
                # Formula: cb = new_short_qty × new_price
                new_short_qty = abs(new_quantity)
                cb = new_short_qty * price
                
            elif new_quantity == 0:
                # Case 5: Full Close Long (END segment, YES)
                # Formula: cb = 0
                cb = Decimal('0')
                
            else:
                # Still long (partial close) - proportional reduction
                sell_qty = qty
                remaining_long = old_quantity - sell_qty
                if remaining_long > 0:
                    cb = cb * (remaining_long / old_quantity)
                else:
                    cb = Decimal('0')

    # Calculate average price
    # Formula: avg_price = abs(cost_basis / quantity) when quantity != 0
    if new_quantity != 0:
        avg_price = abs(cb / new_quantity)
    else:
        avg_price = Decimal('0')
        cb = Decimal('0')

    portfolio_state['cost_basis'][ticker] = cb
    portfolio_state['avg_price'][ticker] = avg_price
    
    # Return as Decimal - NO CONVERSION, NO PRECISION LOSS
    return avg_price, cb

# ---------- Main entry per trade ----------

def process_trade(ticker, action, direction, price, quantity_buy):
    """
    Main function to process a single trade and update portfolio state.
    
    Args:
        ticker (str): Ticker symbol
        action (str): Trade action ('buy', 'sell', 'hold')
        direction (str): Direction type ('long', 'short', 'hold')
        price (float or Decimal): Trade price
        quantity_buy (float, str, or Decimal): Quantity to trade
    
    Returns:
        dict: Row dictionary with all calculated values (ALL as Decimal)
    """
    ticker = str(ticker).strip().upper()
    
    # Convert price to Decimal
    price_dec = Decimal(str(price))
    
    # Normalize quantity input (handles various formats, returns Decimal)
    q_in = normalize_quantity(quantity_buy)

    # Get previous state (already Decimal)
    old_q = portfolio_state['quantities'][ticker]
    old_cb = portfolio_state['cost_basis'][ticker]
    
    # Update quantity and determine current direction
    new_q = calculate_current_quantity_single(ticker, action, q_in, old_q)
    
    # Determine current direction based on new quantity
    if new_q < 0:
        current_direction = 'short'
    elif new_q > 0:
        current_direction = 'long'
    else:  # new_q == 0
        # Retain previous direction type when closing
        if old_q > 0:
            current_direction = 'long'  # Was long, now closed
        elif old_q < 0:
            current_direction = 'short'  # Was short, now closed
        else:
            current_direction = 'hold'  # No previous direction
    
    # Calculate average price and cost basis (returns Decimal - no conversion)
    avg_p, cb = calculate_avg_price_and_cost_basis_single(
        ticker, action, price_dec, q_in, old_q, new_q, old_cb  
    )
    
    # Build row dictionary - ALL VALUES AS DECIMAL (ZERO PRECISION LOSS)
    row = {
        'Ticker': ticker,
        'Side': action.capitalize(),
        'Direction': direction.capitalize(),
        'Current Direction': current_direction.capitalize(),
        'Quantity Buy': q_in,        # Decimal - NO ROUNDING
        'Current Quantity': new_q,   # Decimal - NO ROUNDING
        'Price': price_dec,           # Decimal - NO ROUNDING
        'Avg Price': avg_p,           # Decimal - NO ROUNDING
        'Cost Basis': cb              # Decimal - NO ROUNDING
    }

    # Append row to portfolio DataFrame
    df_row = pd.DataFrame([row], columns=COLUMNS)
    portfolio_state['portfolio_df'] = pd.concat(
        [portfolio_state['portfolio_df'], df_row],
        ignore_index=True
    )

    return row

def add_trade(ticker, action, direction, price, quantity_buy):
    """
    Add a trade to the portfolio and return the updated DataFrame.
    
    Args:
        ticker (str): Ticker symbol
        action (str): Trade action ('buy', 'sell', 'hold')
        direction (str): Direction type ('long', 'short', 'hold')
        price (float or Decimal): Trade price
        quantity_buy (float, str, or Decimal): Quantity to trade
    
    Returns:
        pd.DataFrame: Updated portfolio DataFrame with all trades (ALL Decimal values)
    """
    process_trade(ticker, action, direction, price, quantity_buy)
    return get_portfolio_df()

# ---------- Initialize ----------
reset_portfolio()

In [11]:
# Reset portfolio
reset_portfolio()

add_trade(ticker='btc', action='sell', direction='short', price=86872.7731121207, quantity_buy=1e-05)
add_trade(ticker='btc', action='sell', direction='short', price=86872.7731121207, quantity_buy=1e-05)
add_trade(ticker='btc', action='sell', direction='short', price=86872.7731121207, quantity_buy=1e-05)
add_trade(ticker='btc', action='sell', direction='short', price=86864.5428288392, quantity_buy=0.00104)
add_trade(ticker='btc', action='sell', direction='short', price=86864.5428288392, quantity_buy=0.00104)
add_trade(ticker='btc', action='sell', direction='short', price=86864.5428288392, quantity_buy=0.00104)
add_trade(ticker='btc', action='sell', direction='short', price=86864.5428288392, quantity_buy=0.00104)
add_trade(ticker='btc', action='sell', direction='short', price=86864.5428288392, quantity_buy=0.00104)
add_trade(ticker='btc', action='sell', direction='short', price=86860.0833264288, quantity_buy=0.0017961332779742195)
add_trade(ticker='btc', action='buy', direction='short', price=86855.8629321628, quantity_buy=0.00156)
add_trade(ticker='btc', action='buy', direction='short', price=86855.8629321628, quantity_buy=0.00156)
add_trade(ticker='btc', action='buy', direction='short', price=86855.8629321628, quantity_buy=0.00156)
add_trade(ticker='btc', action='buy', direction='short', price=86855.8629321628, quantity_buy=0.00156)
add_trade(ticker='btc', action='buy', direction='long', price=86855.2888284969, quantity_buy=0.0103174822913711)
add_trade(ticker='btc', action='buy', direction='long', price=86855.2888284969, quantity_buy=0.00571598759206404)
add_trade(ticker='btc', action='sell', direction='short', price=86846.0002552973, quantity_buy=0.006)
add_trade(ticker='btc', action='sell', direction='short', price=86845.9682226498, quantity_buy=0.000535)
add_trade(ticker='btc', action='sell', direction='short', price=86845.9682226498, quantity_buy=0.000453233528959958)
add_trade(ticker='btc', action='buy', direction='long', price=86850.7730363984, quantity_buy=0.0002)
add_trade(ticker='btc', action='buy', direction='long', price=86850.7730363984, quantity_buy=0.0002)
add_trade(ticker='btc', action='buy', direction='long', price=86850.7730363984, quantity_buy=0.0002)
add_trade(ticker='btc', action='buy', direction='long', price=86853.2717518858, quantity_buy=0.00499915611489626)
add_trade(ticker='btc', action='buy', direction='long', price=86876.3645280422, quantity_buy=0.0137616184830761)

# Display
df = get_portfolio_df()
df

,Ticker,Side,Direction,Current Direction,Quantity Buy,Current Quantity,Price,Avg Price,Cost Basis
0,BTC,Sell,Short,Short,0.00001,-0.00001,86872.7731121207,86872.7731121207,0.868727731121207
1,BTC,Sell,Short,Short,0.00001,-0.00002,86872.7731121207,86872.7731121207,1.737455462242414
2,BTC,Sell,Short,Short,0.00001,-0.00003,86872.7731121207,86872.7731121207,2.606183193363621
3,BTC,Sell,Short,Short,0.00104,-0.00107,86864.5428288392,86864.77358444522336448598131,92.94530773535638900000000000
4,BTC,Sell,Short,Short,0.00104,-0.00211,86864.5428288392,86864.65984708490853080568720,183.2844322773491570000000000
5,BTC,Sell,Short,Short,0.00104,-0.00315,86864.5428288392,86864.62121248950000000000,273.6235568193419250000000000
6,BTC,Sell,Short,Short,0.00104,-0.00419,86864.5428288392,86864.60175688178830548926014,363.9626813613346930000000000
7,BTC,Sell,Short,Short,0.00104,-0.00523,86864.5428288392,86864.59003887714359464627151,454.3018059033274610000000000
8,BTC,Sell,Short,Short,0.0017961332779742195,-0.0070261332779742195,86860.0833264288,86863.43796050309594792604678,610.3140920935398691994094023
9,BTC,Buy,Short,Short,0.00156,-0.0054661332779742195,86855.8629321628,86863.43796050309594792604678,474.8071288751550395206447693
